In [148]:
import os
import pandas as pd
import logging
import plotly.graph_objects as go
from tqdm import tqdm
import numpy as np

import cudf as cd
import cupy as cp

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import het_arch, acorr_ljungbox

from scipy.stats import shapiro
from scipy.stats import probplot
from darts.dataprocessing.transformers import Scaler

import darts
from darts.models import RNNModel
from darts.timeseries import TimeSeries as ts

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') 

import warnings
warnings.simplefilter(action='ignore', category= FutureWarning)

RES_PATH = '../../res/'
os.listdir(RES_PATH)

['salinity_2.npy', 'salinity_2.csv', 'water_temp_2.npy', 'water_temp_2.csv']

In [130]:
len(data)

15861

In [146]:
# need to scale all series
sample_sum = np.array([0 for _ in range(264)], dtype='float64')
for idx, (coordinate, sample) in enumerate(data):
    sample_data = np.array(sample)
    
    sample_non_nan = np.nan_to_num(sample_data, nan=0.0)   
    
    sample_data = np.nan_to_num(sample_data, nan=np.mean(sample_non_nan))    
    sample_sum = np.add(sample_sum, sample_data)

    
sample_sum = np.divide(sample_sum, 15681)

df_mean = pd.DataFrame({
    'raw_month': [i for i in range(0, 264)],
    'avg_temp': sample_sum
})

df_mean.head()

,raw_month,avg_temp
0,0,25.776367
1,1,25.282799
2,2,25.292822
3,3,25.646170
4,4,26.625697


In [169]:
transformer = MinMaxScaler()
transformer.fit(df_mean['avg_temp'].values.reshape(-1, 1))

MinMaxScaler()

In [176]:
data = np.load(RES_PATH+'water_temp_2.npy', allow_pickle=True)
data.shape

# data needs to be in a dataframe format with two columns: time (raw_month) and actual value

df = pd.DataFrame()
df['raw_month'] = [i for i in range(0, 264)]
ts_list = []

for idx, (coordinate, sample) in enumerate(data):
    # is this the most memory-efficient? Nope, does it work? yup
    df = pd.DataFrame()
    df['raw_month'] = [i for i in range(0, 264)]    
    norm_data = transformer.transform(sample.reshape(-1, 1))
    
    sample_non_nan = np.nan_to_num(norm_data, nan=0.0) 
    mean = np.mean(sample_non_nan)
    
    df['data'] = norm_data    
    df['raw_month'] = df['raw_month'].astype(int)
    
    ts = ts.from_dataframe(df, time_col='raw_month', value_cols=['data'], fillna_value=mean)
    ts_list.append(ts)
    del df

In [177]:
model = RNNModel(
    model="LSTM",
    hidden_dim=20,
    dropout=0,
    batch_size=16,
    n_epochs=300,
    optimizer_kwargs={"lr": 1e-3},
    model_name="Air_RNN",
    log_tensorboard=True,
    random_state=42,
    training_length=20,
    input_chunk_length=14,
    force_reset=True,
    save_checkpoints=True,
)
model.fit(series=ts_list, verbose=True)

Training: 0it [00:00, ?it/s]

Checking Output

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)

In [42]:
# Read data:
series = AirPassengersDataset().load()

# Create training and validation sets:
train, val = series.split_after(pd.Timestamp("19590101"))

# Normalize the time series (note: we avoid fitting the transformer on the validation set)
transformer = Scaler()
train_transformed = transformer.fit_transform(train)
val_transformed = transformer.transform(val)
series_transformed = transformer.transform(series)

# create month and year covariate series
year_series = datetime_attribute_timeseries(
    pd.date_range(start=series.start_time(), freq=series.freq_str, periods=1000),
    attribute="year",
    one_hot=False,
)
year_series = Scaler().fit_transform(year_series)
month_series = datetime_attribute_timeseries(
    year_series, attribute="month", one_hot=True
)
covariates = year_series.stack(month_series)
cov_train, cov_val = covariates.split_after(pd.Timestamp("19590101"))

month_series

<TimeSeries (DataArray) (time: 1000, component: 12, sample: 1)>
array([[[1.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [1.],
        ...,
...
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [1.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]])
Coordinates:
  * time       (time) datetime64[ns] 1949-01-01 1949-02-01 ... 2032-04-01
  * component  (component) object 'month_1' 'month_2' ... 'month_11' 'month_12'
Dimensions without coordinates: sample